<a href="https://colab.research.google.com/github/bro-ee/Jira-tickets-with-ChatGPT/blob/master/Google_Indexing_API_CSV_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://studiomakoto.it">
  <img src="https://studiomakoto.it/wp-content/uploads/2022/01/Logo-Studio-Makoto-Agenzia-di-Marketing-e-Comunicazione.jpg" alt="Logo Studio Makoto" width="120" height="120">
</a>

**Author**: <a href="https://studiomakoto.it/makoto_member/massimiliano-geraci/">Max Geraci</a> |
<a href="https://twitter.com/maxgeraci1">
  <img src="https://cdn-icons-png.flaticon.com/512/733/733579.png" alt="Twitter Icon" width="20" height="20">
</a>

# Fast Google Indexing via Indexing API

Dear SEO enthusiasts, we're excited to introduce a quick and efficient method to submit your URLs directly to Google's Indexing API for immediate indexing. This approach allows you to send crawl requests straight to Google, making it an ideal solution for platforms like WordPress and Blogger.

Google suggests that the Indexing API is mainly designed for pages embedding either 'JobPosting' or 'BroadcastEvent' within a 'VideoObject'. This makes it particularly useful for websites hosting ephemeral pages such as job postings or live-streamed videos frequently. By pushing individual updates, the Indexing API ensures your content remains up-to-date in search results.

Interestingly, our tests reveal that this method also works flawlessly with other types of websites, extending its benefits beyond the initially intended usage.

## What Can You Do with This Colab Notebook?

This Colab notebook enables you to update or delete URLs from the Google Index and also to check the status of URLs.

To achieve this, follow these steps:

1. Import necessary libraries and upload your JSON key file (Cell 0).

If you want to update or delete URLs:

2. Upload your CSV or Excel file containing the URLs (Cell 1).
3. Choose the request type: 'URL_UPDATED' to update the URL or 'URL_DELETED' to remove the URL from the index.

If you want to check the URLs' indexing status:

4. Upload your CSV or Excel file containing the URLs (Cell 2).
5. The notebook will automatically check the status of each URL and save the results to a CSV file named 'indexed_urls.csv'.

Please note that you can either perform URL updates/deletions or check the status of URLs based on your requirements.

However, to get the Colab notebook to work, you'll need to generate the Indexing API from the Google Cloud Console. Additionally, you'll need to create a service account and add it as an owner in the Search Console account of the website you want to index through the API.

For a detailed step-by-step guide, please refer to the following resources:

1. [Google's official guide on using the Indexing API](https://developers.google.com/search/apis/indexing-api/v3/using-api)
2. [RankMath's blog post on Google Indexing API](https://rankmath.com/blog/google-indexing-api/)

Let's get started!

# Upload your Indexing API credentials

1. Run the cell.
2. When prompted, select and upload your JSON credentials file.

This JSON file should contain the private key for your service account, which you created as part of setting up access to the Indexing API.

For more detailed instructions on creating and managing your service account credentials, please refer to the [Google Search Central developers guide](https://developers.google.com/search/apis/indexing-api/v3/prereqs#oauth).

In [ ]:
#@title **Import Necessary Libraries and Upload JSON Key**
from google.colab import files
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
import pandas as pd
import io

# Upload JSON Key
try:
  uploaded_key = files.upload()
  for key, value in uploaded_key.items():
    json_key_name = key
  print('*'*50)
  print("JSON Key uploaded successfully!")
  print('*'*50)
except:
  print('*'*50)
  print("Error: Please try uploading the JSON Key again!")
  print('*'*50)

# Define scopes and endpoints
SCOPES = ["https://www.googleapis.com/auth/indexing"]
ENDPOINT = "https://indexing.googleapis.com/v3/urlNotifications:publish"
JSON_KEY_FILE = json_key_name

# Authorize the JSON Key
credentials = ServiceAccountCredentials.from_json_keyfile_name(JSON_KEY_FILE, scopes=SCOPES)
http = credentials.authorize(httplib2.Http())
print('*'*50)
print("Credentials authorized successfully!")
print('*'*50)

In [ ]:
#@title **Upload CSV/Excel File and Send Indexing Requests**
# Upload CSV or Excel file
uploaded_file = files.upload()
file_name = list(uploaded_file.keys())[0]
requestType = "URL_UPDATE" #@param ["URL_UPDATE", "URL_DELETE"]

# Read the file
if '.csv' in file_name:
    data = pd.read_csv(io.BytesIO(uploaded_file[file_name]))
elif '.xls' in file_name:
    data = pd.read_excel(io.BytesIO(uploaded_file[file_name]))

# Get list of URLs
urls = data.iloc[:,0].tolist()

# Send indexing requests for each URL and print results
for siteURL in urls:
    content = str({'url':siteURL,'type':requestType})
    response, content = http.request(ENDPOINT, method="POST", body=content)
    output = response['status']

    if output == '200':
        print(f"Successfully Updated {siteURL}")
        print('*'*50)
    else:
        print(f"Error Updating {siteURL}. Error Code: {output}")
        print("Visit Here For More: https://developers.google.com/search/apis/indexing-api/v3/core-errors#api-errors")
        print('*'*50)

In [ ]:
#@title **Upload CSV/Excel File, Check URLs, and Save Results**
# Upload CSV or Excel file
uploaded_file = files.upload()
file_name = list(uploaded_file.keys())[0]

# Read the file
if '.csv' in file_name:
    data = pd.read_csv(io.BytesIO(uploaded_file[file_name]))
elif '.xls' in file_name:
    data = pd.read_excel(io.BytesIO(uploaded_file[file_name]))

# Get list of URLs
urls = data.iloc[:,0].tolist()

# Prepare a DataFrame to store results
results_df = pd.DataFrame(columns=['url', 'is_indexed'])

# Check each URL and save results
for siteURL in urls:
    content = str({'url':siteURL,'type':'URL_UPDATED'})
    response, content = http.request(ENDPOINT, method="POST", body=content)
    output = response['status']

    if output == '200':
        temp_df = pd.DataFrame([{'url': siteURL, 'is_indexed': 'TRUE'}])
    else:
        temp_df = pd.DataFrame([{'url': siteURL, 'is_indexed': 'FALSE'}])

    results_df = pd.concat([results_df, temp_df], ignore_index=True)

# Save results to a CSV file
output_file_name = 'Indexing_status_' + file_name.split('.')[0] + '.csv'
results_df.to_csv(output_file_name, index=False)

# Download the CSV file
files.download(output_file_name)

print(f"Results saved to '{output_file_name}' and downloaded.")